Import the libraries required

In [ ]:
import numpy as np
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd

###### a. Generate 20 data pairs (X, Y) using y = sin(2*pi*X) + 0.1 * N

* Using uniform distribution between 0 and 1 for X
* Sampling N from the normal gaussian distribution
* Using 10 for train and 10 for test 

In [ ]:
np.random.seed(0)

x = np.random.uniform(0.0, 1.0, 20)
N = np.random.normal(0.0, 1.0, 20)
y = np.sin(2*np.pi*x) + (0.1 * N)

x_train = x[:10]
x_test = x[10:]

y_train = y[:10]
y_test = y[10:]


###### b. Using root mean square error, find weights of polynomial regression for order is 0, 1, 3, 9

In [ ]:
class Model(object):

    def __init__(self, pdegree):
        self.pdegree = pdegree
        self.w = []
        self.w0 = torch.ones(1, requires_grad = True)
        self.w.append(self.w0)
        if self.pdegree > 0:
            for i in range(self.pdegree):
                self.w.append(torch.ones(1, requires_grad = True))

    def forward(self, x):
        num = self.w0
        if self.pdegree > 0:
            for i in range(1, len(self.w)):
                num = num + self.w[i]*x**i
        return num

    def loss(self, x_train, y_train):
        y_pred = self.forward(x_train)
        return (y_pred - y_train) * (y_pred - y_train)

    def train(self, epoch = 10):
        print('For polynomial of degree ' + str(self.pdegree))
        for i in range(epoch):
            lossvalue = 0
            for j in range(len(x_train)):
                l = self.loss(x_train[j], y_train[j])
                lossvalue += l
            lossvalue.backward()
            print('Iteration:', i, 'Loss:', lossvalue.data.item())
            self.w0.data = self.w0.data - 0.01*self.w0.grad.data
            if self.pdegree > 0:
                for k in range(1, len(self.w)):
                    self.w[k].data = self.w[k].data - 0.01*self.w[k].grad.data

    
        print('w0 :', self.w0.item())
        if self.pdegree > 0:
            for k in range(1, len(self.w)):
                print('w' + str(k) + ' :', self.w[k].item())

        M = [self.w0.item()]
        if self.pdegree > 0:
            for i in range(1, len(self.w)):
                M.append(self.w[i].item())
 
        return M

degree = [0, 1, 3, 9]
model = []
for d in degree:
      m = Model(d)
      model.append(m.train())

###### c. Display weights in table 

In [ ]:
indexes = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9']

list1 = ["", "", "", "", "", "", "", "", ""]
list2 = ["", "", "", "", "", "", "", ""]
list3 = ["", "", "", "", "", ""]
list4 = []

def makeList(weights, degree, index, mylist):
    for i in range(0, degree+1):
        mylist.insert(i, str(round(weights[index][i], 4)))
    return mylist

weights_new = {
    'M = 0': makeList(model, 0, 0, list1), 
    'M = 1': makeList(model, 1, 1, list2),  
    'M = 3': makeList(model, 3, 2, list3), 
    'M = 9': makeList(model, 9, 3, list4)
    }

weights_df = pd.DataFrame(weights_new, index=indexes)
weights_df